In [1]:
# Load xml-model to mujoco
from mujoco_py import load_model_from_path, MjSim, MjViewer
import matplotlib.pyplot as plt
import numpy as np
import math

from LegModel.legs import LegModel

RUN_STEPS = 100

model = load_model_from_path("../models/dynamic_biped_quad_val.xml") # test.xml / dynamic_4l_VerticalSpine.xml
sim = MjSim(model)

In [2]:
# Run mujoco simulation for one timestep
# relevant for the computation of COM postion in world frame as the actuator can change the position of the bodys
# Initialization with all the actuators set to zeros
# viewer = MjViewer(sim)
# viewer.cam.azimuth = 0
# viewer.cam.lookat[0] += 0.25
# viewer.cam.lookat[1] += -0.5
# viewer.cam.distance = model.stat.extent * 0.5
# viewer.run_speed = 2
# sim_state = sim.get_state()
# sim.set_state(sim_state)

ctrlData = [0, 0, 0, 0, 0, 0, 0, 0]
sim.data.ctrl[:] = ctrlData
sim.step()

# for i in range(500):
#     sim.data.ctrl[:] = ctrlData
#     sim.step()
#     viewer.render()

In [8]:
# Summarize all the important parameters of bodies to a dataframe-table
import pandas
geomName = []
for i in range(sim.model.ngeom):
    geomName.append(sim.model.geom_id2name(i))
    if geomName[-1] == None:
        geomName[-1] = 'None'
df_geom = pandas.DataFrame({"Body Name":geomName})
# df

In [3]:
# Summarize all the important parameters of bodies to a dataframe-table
import pandas
bodyName = []
for i in range(sim.model.nbody):
    bodyName.append(sim.model.body_id2name(i))
    if bodyName[-1] == None:
        bodyName[-1] = 'None'
bodyMass = sim.model.body_mass
body_COM_pos = sim.data.xipos
df = pandas.DataFrame({"Body Name":bodyName, "Body Mass":bodyMass})
df['Body COM-based Inertia in x-axis'] = sim.model.body_inertia[:,-1]
df['Body COM Position'] = np.around(body_COM_pos, decimals=3).tolist()
# df

In [ ]:
# calculated the key parameters based on dynamic model with rigid spine
body_COM_pos_y = body_COM_pos[:,1] # bodies pos in y-axis
body_COM_pos_z = body_COM_pos[:,2] # bodies pos in z-axis
body_mass = sum(bodyMass) # whole body mass
COM_y = sum(body_COM_pos_y*bodyMass) / sum(bodyMass) # whole body COM position in y-axis
COM_z = sum(body_COM_pos_z*bodyMass) / sum(bodyMass) # whole body COM position in z-axis

i_xx_i = sim.model.body_inertia[:,-1] # com-based bodies moment of inertia around x-axis
l_i_COM_quad = (body_COM_pos_y-COM_y)**2 + (body_COM_pos_z-COM_z)**2 # quadrat distance between each body COM pos and whole body COM pos
i_xx_sum = sum(i_xx_i) + sum(bodyMass*l_i_COM_quad) #com-based whole body moment of inertia around x-axis
print(i_xx_sum)

In [4]:
# Summarize all the important parameters of sites to a dataframe-table
import pandas
siteName = []
for i in range(sim.model.nsite):
    siteName.append(sim.model.site_id2name(i))
    if siteName[-1] == None:
        siteName[-1] = 'None'
site_pos = sim.data.site_xpos
df2 = pandas.DataFrame({"Site Name":siteName})
df2['Site Position'] = np.around(site_pos, decimals=3).tolist()
# df2

In [5]:
# COM pos of each subtree
# np.around(sim.data.subtree_com, decimals=3)
print("Body COM pos: ", sim.data.subtree_com[0,:])
# print("Body Contact point pos: ", sim.data.contact[-1].geom1)
footendPos_fl = site_pos[siteName.index("ankle_fl")]
footendPos_fr = site_pos[siteName.index("ankle_fr")]
footendPos_rl = site_pos[siteName.index("ankle_rl")]
footendPos_rr = site_pos[siteName.index("ankle_rr")]
print("Left Front Foot Pos: ", footendPos_fl)
print("Right Front Foot Pos: ", footendPos_fr)
print("Left Hint Foot Pos: ", footendPos_rl)
print("Right Hint Foot Pos: ", footendPos_rr)
print("--------------------------------------------------")
legLinkPos_fl = site_pos[siteName.index("leg_link_fl")]
legLinkPos_fr = site_pos[siteName.index("leg_link_fr")]
print("Left Leg Link Pos: ", legLinkPos_fl)
print("Right Leg Link Pos: ", legLinkPos_fr)

Body COM pos:  [ 6.17314728e-13 -9.00726911e-04  4.19682762e-02]
Left Front Foot Pos:  [ 0.0361     -0.03155802  0.0050382 ]
Right Front Foot Pos:  [-0.0361     -0.03155802  0.0050382 ]
Left Hint Foot Pos:  [0.0361     0.03544198 0.0050382 ]
Right Hint Foot Pos:  [-0.0361      0.03544198  0.0050382 ]
--------------------------------------------------
Left Leg Link Pos:  [ 0.0361 -0.0455  0.0356]
Right Leg Link Pos:  [-0.0361 -0.0455  0.0356]


In [ ]:
# Blocks below not relevant

In [20]:
def footendForce_2_torque(force_d, q1, q2):
    j11 = - (31*math.sin(q1))/300 - (7*math.cos(math.acos((25*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2))) + math.asin(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)/(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)))*(((25*((31*math.cos(q1)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/500 + (31*math.sin(q1)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/500))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)) - (25*((31*math.cos(q1)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/500 + (31*math.sin(q1)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/500)*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000))/(4*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(3/2)))/(1 - (625*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000)**2)/(4*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + 4*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2))**(1/2) - ((31*math.sin(q1))/(1000*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)) - (((31*math.cos(q1)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/500 + (31*math.sin(q1)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/500)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(3/2)))/(1 - ((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2/(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2))**(1/2)))/75
    j12 = (413*math.cos(q2))/15000 + (7*math.cos(math.acos((25*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2))) + math.asin(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)/(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)))*(((25*((59*math.cos(q2)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/2500 - (59*math.sin(q2)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/2500))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)) - (25*((59*math.cos(q2)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/2500 - (59*math.sin(q2)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/2500)*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000))/(4*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(3/2)))/(1 - (625*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000)**2)/(4*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + 4*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2))**(1/2) - ((59*math.cos(q2))/(5000*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)) - (((59*math.cos(q2)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/2500 - (59*math.sin(q2)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/2500)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(3/2)))/(1 - ((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2/(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2))**(1/2)))/75
    j21 = (31*math.cos(q1))/300 - (7*math.sin(math.acos((25*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2))) + math.asin(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)/(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)))*(((25*((31*math.cos(q1)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/500 + (31*math.sin(q1)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/500))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)) - (25*((31*math.cos(q1)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/500 + (31*math.sin(q1)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/500)*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000))/(4*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(3/2)))/(1 - (625*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000)**2)/(4*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + 4*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2))**(1/2) - ((31*math.sin(q1))/(1000*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)) - (((31*math.cos(q1)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/500 + (31*math.sin(q1)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/500)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(3/2)))/(1 - ((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2/(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2))**(1/2)))/75
    j22 = (413*math.sin(q2))/15000 + (7*math.sin(math.acos((25*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2))) + math.asin(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)/(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)))*(((25*((59*math.cos(q2)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/2500 - (59*math.sin(q2)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/2500))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)) - (25*((59*math.cos(q2)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/2500 - (59*math.sin(q2)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/2500)*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000))/(4*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(3/2)))/(1 - (625*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2 + 11/8000)**2)/(4*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + 4*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2))**(1/2) - ((59*math.cos(q2))/(5000*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(1/2)) - (((59*math.cos(q2)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/2500 - (59*math.sin(q2)*((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625))/2500)*((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000))/(2*(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2)**(3/2)))/(1 - ((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2/(((31*math.cos(q1))/1000 + (59*math.sin(q2))/5000)**2 + ((59*math.cos(q2))/5000 - (31*math.sin(q1))/1000 + 8/625)**2))**(1/2)))/75
    j = np.array([[j11, j12], [j21, j22]])
    j_T = np.transpose(j)
    torque = np.dot(j_T, force_d)

    return torque

q1 = - 17.3175/180*math.pi
q2 = - 29.6815/180*math.pi
force_d = np.array([[0], [1]])
torque = -footendForce_2_torque(force_d, q1, q2)
print(torque)

[[-0.04226071]
 [ 0.01892811]]


In [23]:
import math
import numpy as np

# a = np.array([[1,2],[0,1]])
# b = np.array([[2],[1]])
# c = np.dot(a,b)
# print(a, "\n", b, "\n", c)
# d = np.append(np.transpose(c), np.transpose(c), 1)[0]
# print(d)
ini = np.empty((0,2))
a = np.array([[1,2],[0,1]])
c = np.array([[3, 4]])
ini = np.append(ini, c, 0)
ini = np.append(ini, c, 0)
ini

array([[3., 4.],
       [3., 4.]])

In [7]:
ctrlData = np.array([-0.00830482,  0.00271565, -0.00830482,  0.00271565])
ctrlData = np.hstack((ctrlData, ctrlData))
ctrlData

array([-0.00830482,  0.00271565, -0.00830482,  0.00271565, -0.00830482,
        0.00271565, -0.00830482,  0.00271565])